# Lab 3.6.2: ReAct Agent - SOLUTIONS

**Complete solutions with explanations and alternative approaches**

---

## Setup

In [ ]:
import re
import json
from typing import Dict, Any, List, Optional, Callable
from dataclasses import dataclass, field
from enum import Enum
from datetime import datetime

print("Setup complete!")

---

## Exercise 1 Solution: Build a Complete ReAct Agent

**Task**: Implement a ReAct agent from scratch that can handle multi-step reasoning.

In [ ]:
class AgentState(Enum):
    """Agent execution states."""
    THINKING = "thinking"
    ACTING = "acting"
    OBSERVING = "observing"
    FINISHED = "finished"
    ERROR = "error"


@dataclass
class ReActStep:
    """A single step in ReAct execution."""
    step_number: int
    thought: str
    action: Optional[str] = None
    action_input: Optional[str] = None
    observation: Optional[str] = None
    state: AgentState = AgentState.THINKING


@dataclass
class AgentConfig:
    """Configuration for the ReAct agent."""
    max_steps: int = 10
    verbose: bool = True
    retry_on_error: bool = True
    max_retries: int = 2


class ReActAgent:
    """
    Production-ready ReAct (Reasoning + Acting) agent.
    
    Features:
    - Multi-step reasoning
    - Tool execution with error handling
    - Execution trace for debugging
    - Configurable behavior
    """
    
    SYSTEM_PROMPT = """You are a helpful assistant that uses tools to answer questions.

You have access to the following tools:
{tools_description}

Use this format:

Question: the input question you must answer
Thought: reason about what to do step by step
Action: the tool name to use (one of: {tool_names})
Action Input: the input for the tool
Observation: the result from the tool
... (repeat Thought/Action/Action Input/Observation as needed)
Thought: I now know the final answer
Final Answer: the final answer to the question

Begin!
"""
    
    def __init__(self, tools: Dict[str, Callable], config: Optional[AgentConfig] = None):
        """
        Initialize ReAct agent.
        
        Args:
            tools: Dictionary of tool_name -> callable
            config: Agent configuration
        """
        self.tools = tools
        self.config = config or AgentConfig()
        self.execution_trace: List[ReActStep] = []
        self.current_state = AgentState.THINKING
    
    def _get_tools_description(self) -> str:
        """Generate tools description for prompt."""
        descriptions = []
        for name, func in self.tools.items():
            doc = func.__doc__ or "No description"
            descriptions.append(f"- {name}: {doc.strip()}")
        return "\n".join(descriptions)
    
    def _parse_llm_output(self, output: str) -> Dict[str, Any]:
        """
        Parse LLM output to extract thought, action, and final answer.
        """
        result = {
            "thought": None,
            "action": None,
            "action_input": None,
            "final_answer": None,
        }
        
        # Extract thought
        thought_match = re.search(r'Thought:\s*(.+?)(?=Action:|Final Answer:|$)', output, re.DOTALL)
        if thought_match:
            result["thought"] = thought_match.group(1).strip()
        
        # Extract action
        action_match = re.search(r'Action:\s*(.+?)(?=Action Input:|$)', output, re.DOTALL)
        if action_match:
            result["action"] = action_match.group(1).strip()
        
        # Extract action input
        input_match = re.search(r'Action Input:\s*(.+?)(?=Observation:|Thought:|$)', output, re.DOTALL)
        if input_match:
            result["action_input"] = input_match.group(1).strip()
        
        # Extract final answer
        final_match = re.search(r'Final Answer:\s*(.+?)$', output, re.DOTALL)
        if final_match:
            result["final_answer"] = final_match.group(1).strip()
        
        return result
    
    def _execute_tool(self, action: str, action_input: str) -> str:
        """Execute a tool and return observation."""
        if action not in self.tools:
            return f"Error: Unknown tool '{action}'. Available: {list(self.tools.keys())}"
        
        try:
            result = self.tools[action](action_input)
            return str(result)
        except Exception as e:
            return f"Error executing {action}: {str(e)}"
    
    def _simulate_llm_response(self, prompt: str, history: str) -> str:
        """
        Simulate LLM response for demonstration.
        In production, this would call the actual LLM.
        """
        # Simple rule-based simulation for demo
        if "capital" in prompt.lower() and "france" in prompt.lower():
            if "Observation:" not in history:
                return """Thought: I need to search for the capital of France.
Action: search
Action Input: capital of France"""
            else:
                return """Thought: I now know the final answer based on my search.
Final Answer: The capital of France is Paris."""
        
        if "weather" in prompt.lower():
            if "Observation:" not in history:
                return """Thought: I need to check the current weather.
Action: weather
Action Input: current location"""
            else:
                return """Thought: I have the weather information.
Final Answer: Based on the weather data, it's currently sunny and 72F."""
        
        if "calculate" in prompt.lower() or any(op in prompt for op in ['+', '-', '*', '/', 'plus', 'minus']):
            # Extract numbers for calculation
            numbers = re.findall(r'\d+', prompt)
            if len(numbers) >= 2 and "Observation:" not in history:
                return f"""Thought: I need to perform a calculation.
Action: calculate
Action Input: {numbers[0]} + {numbers[1]}"""
            else:
                return """Thought: I have completed the calculation.
Final Answer: The result of the calculation is shown in my observation."""
        
        return """Thought: I'll try to answer directly.
Final Answer: I don't have enough information to answer this question accurately."""
    
    def run(self, question: str) -> str:
        """
        Run the ReAct agent on a question.
        
        Args:
            question: The user's question
            
        Returns:
            The final answer
        """
        self.execution_trace = []
        self.current_state = AgentState.THINKING
        
        # Build system prompt
        system_prompt = self.SYSTEM_PROMPT.format(
            tools_description=self._get_tools_description(),
            tool_names=", ".join(self.tools.keys())
        )
        
        # Initialize conversation
        history = f"Question: {question}\n"
        
        for step_num in range(1, self.config.max_steps + 1):
            # Get LLM response
            llm_output = self._simulate_llm_response(question, history)
            
            # Parse output
            parsed = self._parse_llm_output(llm_output)
            
            # Create step record
            step = ReActStep(
                step_number=step_num,
                thought=parsed["thought"],
                action=parsed["action"],
                action_input=parsed["action_input"],
            )
            
            if self.config.verbose:
                print(f"\n--- Step {step_num} ---")
                print(f"Thought: {step.thought}")
            
            # Check for final answer
            if parsed["final_answer"]:
                step.state = AgentState.FINISHED
                self.execution_trace.append(step)
                self.current_state = AgentState.FINISHED
                
                if self.config.verbose:
                    print(f"Final Answer: {parsed['final_answer']}")
                
                return parsed["final_answer"]
            
            # Execute action
            if parsed["action"]:
                step.state = AgentState.ACTING
                
                if self.config.verbose:
                    print(f"Action: {step.action}")
                    print(f"Action Input: {step.action_input}")
                
                observation = self._execute_tool(parsed["action"], parsed["action_input"])
                step.observation = observation
                step.state = AgentState.OBSERVING
                
                if self.config.verbose:
                    print(f"Observation: {observation}")
                
                # Update history
                history += f"""Thought: {step.thought}
Action: {step.action}
Action Input: {step.action_input}
Observation: {observation}
"""
            
            self.execution_trace.append(step)
        
        # Max steps reached
        self.current_state = AgentState.ERROR
        return "I couldn't find an answer within the allowed steps."
    
    def get_trace(self) -> List[Dict]:
        """Get execution trace as list of dicts."""
        return [
            {
                "step": s.step_number,
                "thought": s.thought,
                "action": s.action,
                "action_input": s.action_input,
                "observation": s.observation,
                "state": s.state.value,
            }
            for s in self.execution_trace
        ]


# Define tools
def search(query: str) -> str:
    """Search for information on a topic."""
    # Simulated search results
    if "france" in query.lower() and "capital" in query.lower():
        return "Paris is the capital of France. It is known as the City of Light."
    return f"Search results for: {query}"

def calculate(expression: str) -> str:
    """Calculate a mathematical expression."""
    try:
        result = eval(expression, {"__builtins__": {}}, {})
        return str(result)
    except:
        return "Error in calculation"

def weather(location: str) -> str:
    """Get weather for a location."""
    return f"Weather in {location}: Sunny, 72F, humidity 45%"


# Test the agent
print("=" * 60)
print("REACT AGENT SOLUTION")
print("=" * 60)

tools = {
    "search": search,
    "calculate": calculate,
    "weather": weather,
}

agent = ReActAgent(tools, AgentConfig(verbose=True))

# Test questions
questions = [
    "What is the capital of France?",
    "What is 25 plus 37?",
]

for q in questions:
    print(f"\n{'='*60}")
    print(f"Question: {q}")
    answer = agent.run(q)
    print(f"\nFinal Result: {answer}")

---

## Exercise 2 Solution: Agent with Memory

**Task**: Extend the agent to maintain conversation history across multiple interactions.

In [ ]:
from collections import deque
from typing import Deque, Tuple


@dataclass
class MemoryConfig:
    """Configuration for agent memory."""
    max_history: int = 10
    include_observations: bool = True
    summarize_old: bool = False


class ConversationMemory:
    """
    Memory system for ReAct agent.
    
    Features:
    - Fixed-size conversation history
    - Entity tracking
    - Context summarization
    """
    
    def __init__(self, config: Optional[MemoryConfig] = None):
        self.config = config or MemoryConfig()
        self.history: Deque[Tuple[str, str]] = deque(maxlen=self.config.max_history)
        self.entities: Dict[str, Any] = {}
        self.context_summary: str = ""
    
    def add_interaction(self, question: str, answer: str) -> None:
        """Add a Q&A interaction to memory."""
        self.history.append((question, answer))
        self._extract_entities(question, answer)
    
    def _extract_entities(self, question: str, answer: str) -> None:
        """Extract and store named entities from conversation."""
        # Simple entity extraction (in production, use NER)
        text = f"{question} {answer}"
        
        # Extract capitalized words as potential entities
        words = re.findall(r'\b[A-Z][a-z]+(?:\s+[A-Z][a-z]+)*\b', text)
        for word in words:
            if word not in ['The', 'What', 'How', 'When', 'Where', 'Why', 'Is', 'Are']:
                self.entities[word] = self.entities.get(word, 0) + 1
    
    def get_context(self, max_turns: int = 5) -> str:
        """Get recent conversation context."""
        recent = list(self.history)[-max_turns:]
        
        if not recent:
            return ""
        
        context_parts = ["Previous conversation:"]
        for q, a in recent:
            context_parts.append(f"Q: {q}")
            context_parts.append(f"A: {a}")
        
        return "\n".join(context_parts)
    
    def get_relevant_entities(self, query: str) -> List[str]:
        """Get entities relevant to query."""
        query_lower = query.lower()
        return [
            entity for entity in self.entities.keys()
            if entity.lower() in query_lower or 
               any(word in entity.lower() for word in query_lower.split())
        ]
    
    def clear(self) -> None:
        """Clear all memory."""
        self.history.clear()
        self.entities.clear()
        self.context_summary = ""


class MemoryReActAgent(ReActAgent):
    """
    ReAct agent with conversation memory.
    """
    
    def __init__(self, tools: Dict[str, Callable], 
                 config: Optional[AgentConfig] = None,
                 memory_config: Optional[MemoryConfig] = None):
        super().__init__(tools, config)
        self.memory = ConversationMemory(memory_config)
    
    def run(self, question: str) -> str:
        """Run agent with memory context."""
        # Get context from memory
        context = self.memory.get_context()
        
        # Enhance question with context if relevant
        relevant_entities = self.memory.get_relevant_entities(question)
        
        if self.config.verbose and context:
            print("\n--- Memory Context ---")
            print(context)
            if relevant_entities:
                print(f"Relevant entities: {relevant_entities}")
        
        # Run parent agent
        answer = super().run(question)
        
        # Store in memory
        self.memory.add_interaction(question, answer)
        
        return answer


# Test memory agent
print("=" * 60)
print("MEMORY REACT AGENT SOLUTION")
print("=" * 60)

memory_agent = MemoryReActAgent(
    tools,
    AgentConfig(verbose=True),
    MemoryConfig(max_history=10)
)

# Multi-turn conversation
conversation = [
    "What is the capital of France?",
    "What's the weather there?",  # "there" refers to Paris
]

for q in conversation:
    print(f"\n{'='*60}")
    print(f"User: {q}")
    answer = memory_agent.run(q)
    print(f"\nAgent: {answer}")

# Show memory state
print(f"\n--- Memory State ---")
print(f"Entities tracked: {memory_agent.memory.entities}")
print(f"History length: {len(memory_agent.memory.history)}")

---

## Exercise 3 Solution: Self-Correcting Agent

**Task**: Build an agent that can recognize and recover from errors.

In [ ]:
class ErrorType(Enum):
    """Types of errors the agent can handle."""
    TOOL_NOT_FOUND = "tool_not_found"
    TOOL_EXECUTION = "tool_execution"
    INVALID_INPUT = "invalid_input"
    TIMEOUT = "timeout"
    HALLUCINATION = "hallucination"


@dataclass
class ErrorRecord:
    """Record of an error and recovery attempt."""
    error_type: ErrorType
    error_message: str
    recovery_action: str
    successful: bool
    timestamp: float = field(default_factory=lambda: datetime.now().timestamp())


class SelfCorrectingAgent(ReActAgent):
    """
    Agent with self-correction capabilities.
    
    Features:
    - Error detection
    - Automatic retry with different strategies
    - Learning from errors
    """
    
    def __init__(self, tools: Dict[str, Callable], config: Optional[AgentConfig] = None):
        super().__init__(tools, config)
        self.error_log: List[ErrorRecord] = []
        self.recovery_strategies: Dict[ErrorType, Callable] = {
            ErrorType.TOOL_NOT_FOUND: self._recover_tool_not_found,
            ErrorType.TOOL_EXECUTION: self._recover_tool_execution,
            ErrorType.INVALID_INPUT: self._recover_invalid_input,
        }
    
    def _detect_error(self, observation: str) -> Optional[ErrorType]:
        """Detect error type from observation."""
        observation_lower = observation.lower()
        
        if "unknown tool" in observation_lower:
            return ErrorType.TOOL_NOT_FOUND
        if "error executing" in observation_lower:
            return ErrorType.TOOL_EXECUTION
        if "invalid" in observation_lower or "could not parse" in observation_lower:
            return ErrorType.INVALID_INPUT
        
        return None
    
    def _recover_tool_not_found(self, action: str, action_input: str) -> Tuple[str, str]:
        """Recover from tool not found error."""
        # Try to find similar tool
        similar = None
        action_lower = action.lower()
        
        for tool_name in self.tools.keys():
            if tool_name.lower() in action_lower or action_lower in tool_name.lower():
                similar = tool_name
                break
        
        if similar:
            return similar, action_input
        
        # Fall back to search if available
        if "search" in self.tools:
            return "search", action_input
        
        return action, action_input  # No recovery possible
    
    def _recover_tool_execution(self, action: str, action_input: str) -> Tuple[str, str]:
        """Recover from tool execution error."""
        # Try to simplify input
        simplified = action_input.strip()
        
        # Remove special characters
        simplified = re.sub(r'[^\w\s\d+-/*().]', '', simplified)
        
        return action, simplified
    
    def _recover_invalid_input(self, action: str, action_input: str) -> Tuple[str, str]:
        """Recover from invalid input error."""
        # Try to extract just the core query
        words = action_input.split()
        if len(words) > 5:
            # Take first 5 words
            return action, " ".join(words[:5])
        
        return action, action_input
    
    def _execute_tool(self, action: str, action_input: str) -> str:
        """Execute tool with error recovery."""
        for attempt in range(self.config.max_retries + 1):
            observation = super()._execute_tool(action, action_input)
            
            error_type = self._detect_error(observation)
            
            if error_type is None:
                # No error, return result
                return observation
            
            if attempt < self.config.max_retries:
                # Try recovery
                if error_type in self.recovery_strategies:
                    recovery_fn = self.recovery_strategies[error_type]
                    new_action, new_input = recovery_fn(action, action_input)
                    
                    if self.config.verbose:
                        print(f"  [Recovery] {error_type.value}: {action} -> {new_action}")
                    
                    action, action_input = new_action, new_input
                    
                    # Log recovery attempt
                    self.error_log.append(ErrorRecord(
                        error_type=error_type,
                        error_message=observation,
                        recovery_action=f"{new_action}({new_input})",
                        successful=False,  # Will update if recovery works
                    ))
        
        return observation
    
    def get_error_stats(self) -> Dict[str, Any]:
        """Get error statistics."""
        if not self.error_log:
            return {"total_errors": 0}
        
        by_type = {}
        for record in self.error_log:
            by_type[record.error_type.value] = by_type.get(record.error_type.value, 0) + 1
        
        return {
            "total_errors": len(self.error_log),
            "by_type": by_type,
            "recovery_rate": sum(1 for r in self.error_log if r.successful) / len(self.error_log),
        }


# Test self-correcting agent
print("=" * 60)
print("SELF-CORRECTING AGENT SOLUTION")
print("=" * 60)

correcting_agent = SelfCorrectingAgent(
    tools,
    AgentConfig(verbose=True, max_retries=2)
)

# The agent will handle typos and errors
print("\nTesting error recovery...")
result = correcting_agent._execute_tool("serach", "capital of France")  # Typo in 'search'
print(f"Result: {result}")

print(f"\nError stats: {correcting_agent.get_error_stats()}")

---

## Challenge Solution: Agent Orchestrator

**Task**: Build a system that can coordinate multiple specialized agents.

In [ ]:
from typing import NamedTuple


class AgentSpec(NamedTuple):
    """Specification for a specialized agent."""
    name: str
    description: str
    capabilities: List[str]


class AgentOrchestrator:
    """
    Orchestrator for coordinating multiple specialized agents.
    
    Features:
    - Agent routing based on query type
    - Parallel execution when possible
    - Result aggregation
    - Fallback handling
    """
    
    def __init__(self):
        self.agents: Dict[str, Tuple[AgentSpec, ReActAgent]] = {}
        self.routing_rules: Dict[str, List[str]] = {}  # keyword -> agent_names
    
    def register_agent(self, spec: AgentSpec, agent: ReActAgent) -> None:
        """Register a specialized agent."""
        self.agents[spec.name] = (spec, agent)
        
        # Auto-create routing rules from capabilities
        for capability in spec.capabilities:
            if capability not in self.routing_rules:
                self.routing_rules[capability] = []
            self.routing_rules[capability].append(spec.name)
    
    def route_query(self, query: str) -> List[str]:
        """Determine which agents should handle a query."""
        query_lower = query.lower()
        matching_agents = set()
        
        for keyword, agent_names in self.routing_rules.items():
            if keyword.lower() in query_lower:
                matching_agents.update(agent_names)
        
        # If no matches, return all agents (let them try)
        if not matching_agents:
            return list(self.agents.keys())
        
        return list(matching_agents)
    
    def run(self, query: str, max_agents: int = 3) -> Dict[str, Any]:
        """
        Run query through appropriate agents.
        
        Args:
            query: User query
            max_agents: Maximum agents to consult
            
        Returns:
            Aggregated results from agents
        """
        # Route query
        agent_names = self.route_query(query)[:max_agents]
        
        results = {
            "query": query,
            "agents_consulted": agent_names,
            "responses": {},
            "consensus": None,
        }
        
        # Run each agent
        for name in agent_names:
            spec, agent = self.agents[name]
            try:
                answer = agent.run(query)
                results["responses"][name] = {
                    "answer": answer,
                    "success": True,
                }
            except Exception as e:
                results["responses"][name] = {
                    "answer": None,
                    "success": False,
                    "error": str(e),
                }
        
        # Find consensus (simple: most common answer)
        answers = [r["answer"] for r in results["responses"].values() if r["success"]]
        if answers:
            # Take first successful answer as consensus for simplicity
            results["consensus"] = answers[0]
        
        return results
    
    def list_agents(self) -> List[Dict]:
        """List all registered agents."""
        return [
            {
                "name": spec.name,
                "description": spec.description,
                "capabilities": spec.capabilities,
            }
            for spec, _ in self.agents.values()
        ]


# Create specialized agents
math_tools = {"calculate": calculate}
search_tools = {"search": search}
weather_tools = {"weather": weather}

math_agent = ReActAgent(math_tools, AgentConfig(verbose=False))
research_agent = ReActAgent(search_tools, AgentConfig(verbose=False))
weather_agent = ReActAgent(weather_tools, AgentConfig(verbose=False))

# Create orchestrator
orchestrator = AgentOrchestrator()

orchestrator.register_agent(
    AgentSpec("math", "Handles mathematical calculations", ["calculate", "math", "number", "sum", "add"]),
    math_agent
)
orchestrator.register_agent(
    AgentSpec("research", "Searches for information", ["search", "find", "what is", "who", "capital"]),
    research_agent
)
orchestrator.register_agent(
    AgentSpec("weather", "Provides weather information", ["weather", "temperature", "forecast"]),
    weather_agent
)

# Test orchestrator
print("=" * 60)
print("AGENT ORCHESTRATOR SOLUTION")
print("=" * 60)

print("\nRegistered agents:")
for agent_info in orchestrator.list_agents():
    print(f"  - {agent_info['name']}: {agent_info['description']}")

test_queries = [
    "What is 25 + 37?",
    "What is the capital of France?",
    "What's the weather today?",
]

for query in test_queries:
    print(f"\n--- Query: {query} ---")
    result = orchestrator.run(query)
    print(f"Routed to: {result['agents_consulted']}")
    print(f"Consensus: {result['consensus']}")

---

## Performance Comparison

In [ ]:
import time

def benchmark_agent(agent, query, iterations=5):
    """Benchmark agent performance."""
    times = []
    for _ in range(iterations):
        start = time.perf_counter()
        agent.run(query)
        times.append(time.perf_counter() - start)
    return {
        "avg_ms": sum(times) / len(times) * 1000,
        "min_ms": min(times) * 1000,
        "max_ms": max(times) * 1000,
    }

print("Performance Benchmarks (simulated LLM):")
print("="*50)

# Basic agent
basic_agent = ReActAgent(tools, AgentConfig(verbose=False))
basic_stats = benchmark_agent(basic_agent, "What is 5 + 5?")
print(f"Basic ReAct: {basic_stats['avg_ms']:.2f}ms avg")

# Memory agent
mem_agent = MemoryReActAgent(tools, AgentConfig(verbose=False))
mem_stats = benchmark_agent(mem_agent, "What is 5 + 5?")
print(f"Memory ReAct: {mem_stats['avg_ms']:.2f}ms avg")

# Self-correcting agent
corr_agent = SelfCorrectingAgent(tools, AgentConfig(verbose=False))
corr_stats = benchmark_agent(corr_agent, "What is 5 + 5?")
print(f"Self-Correcting: {corr_stats['avg_ms']:.2f}ms avg")

---

## Key Takeaways

1. **ReAct Pattern**: Think -> Act -> Observe provides structured reasoning
2. **Memory**: Conversation history improves multi-turn interactions
3. **Error Recovery**: Agents should gracefully handle and recover from errors
4. **Orchestration**: Complex tasks benefit from specialized agents working together
5. **Tracing**: Execution traces are essential for debugging and improvement